In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
import json

In [4]:
rdd=sc.textFile("/home/deepika/Downloads/netflix_titles.json")

In [5]:
data= rdd.map(lambda x: json.loads(x))

In [6]:
from pyspark.storagelevel import StorageLevel
data.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[2] at RDD at PythonRDD.scala:53

In [7]:
data.take(1)

[{'show_id': 's1',
  'type': 'TV Show',
  'title': '3%',
  'director': '',
  'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
  'country': 'Brazil',
  'date_added': 'August 14, 2020',
  'release_year': '2020',
  'rating': 'TV-MA',
  'duration': '4 Seasons',
  'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
  'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}]


# how many titles actor/actress appeared in?

In [8]:
titles= data.flatMap(lambda x: (x['cast'].split(",")))

In [9]:
gap=titles.filter(lambda x: x !='')

In [10]:
result= gap.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [11]:
result.take(4)

[(' Vaneza Oliveira', 1),
 (' Viviane Porto', 1),
 (' Mel Fronckowiak', 1),
 (' Celso Frateschi', 1)]

In [12]:
import os
os.environ["PYTHONHASHSEED"]=str(232)

In [13]:
result.lookup(' Jim Sturgess')

[4]

# How many movies were released in 2008?

In [14]:
movies=data.filter(lambda x:x['release_year']=='2008')

In [15]:
movies.count()

125

In [16]:
movies.take(1)

[{'show_id': 's5',
  'type': 'Movie',
  'title': '21',
  'director': 'Robert Luketic',
  'cast': 'Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, Jacob Pitts, Laurence Fishburne, Jack McGee, Josh Gad, Sam Golzari, Helen Carey, Jack Gilpin',
  'country': 'United States',
  'date_added': 'January 1, 2020',
  'release_year': '2008',
  'rating': 'PG-13',
  'duration': '123 min',
  'listed_in': 'Dramas',
  'description': 'A brilliant group of students become card-counting experts with the intent of swindling millions out of Las Vegas casinos by playing blackjack.'}]

# List all the movies whose duration is greater than 100 mins ?

In [17]:
pairs = data.map(lambda x: (int(x['duration'].split(" ")[0]), x['title']))

In [18]:
pairs.take(2)

[(4, '3%'), (93, '7:19')]

In [19]:
duration_above_hundred= pairs.filter(lambda x: x[0] >100).groupByKey().mapValues(list).filter(lambda x: x[0]!='')

In [31]:
duration_above_hundred.take(1)

[(118,
  ['706',
   'Ant-Man and the Wasp',
   'Bathinda Express',
   'Bright',
   'Chalay Thay Saath',
   'Chupan Chupai',
   'Colkatay Columbus',
   'Daagdi Chaawl',
   'Dilan 1991',
   'Dolemite Is My Name',
   'Fear and Loathing in Las Vegas',
   'Fierce',
   'Filosofi Kopi The Movie',
   'Four Weddings and a Funeral',
   'Gelo',
   'Helios',
   'Heneral Luna',
   'High & Low The Movie 3 / Final Mission',
   'History of Joy',
   'Hitch',
   'Hokkabaz',
   'Incredibles 2 (Spanish Version)',
   'Kartini: Princess of Java',
   'Kate & Leopold',
   'Lagos Real Fake Life',
   'Liefling',
   "Logan's Run",
   'Macchli Jal Ki Rani Hai',
   'Memoir of a Murderer',
   'Nightcrawler',
   'Oye Lucky! Lucky Oye!',
   'Phir Se',
   'Revelations',
   "River's Edge",
   'Room',
   'Saath Saath',
   'Saavat',
   'Saheb Bibi Golaam',
   'The Black Godfather',
   'The Book of Eli',
   'The Breakup Playlist',
   "The Butterfly's Dream",
   'The Command',
   'The Incredibles 2',
   "The King's Speech"

In [21]:
duration_above_hundred.count()

107

# List movies played by “Kareena Kapoor” ?

In [22]:
kareena = data.map(lambda x: (x['cast'].split(",")[0], x['title']))

In [23]:
kareena.take(5)

[('João Miguel', '3%'),
 ('Demián Bichir', '7:19'),
 ('Tedd Chan', '23:59'),
 ('Elijah Wood', '9'),
 ('Jim Sturgess', '21')]

In [24]:
kareena.groupByKey().mapValues(list).filter(lambda x: x[0]!='').take(5)

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('Omar Epps', ['3022']),
 ('Sadiq Daba', ['Oct-01'])]

In [25]:
kareena1=[]
kareenaa= kareena.groupByKey().map(lambda x : (x[0], list(x[1]))).filter(lambda x: x[0]!='')

In [26]:
kareenaa.take(2)

[('Tedd Chan', ['23:59']), ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken'])]

In [27]:
movie_list_by_kareena_kapoor=kareenaa.lookup("Kareena Kapoor")

In [28]:
movie_list_by_kareena_kapoor

[['Chameli', 'Ek Main Aur Ekk Tu', 'Heroine', 'Kurbaan', 'Satyagraha']]

In [30]:
movie_list_by_kareena_kapoor.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movie_list_by_kareena_kapoor/")

In [ ]:
duration_above_hundred.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/duration_above_hundred/")

In [ ]:
movies.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movies_in_2008/")

In [ ]:
result.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/titles_actor_actress_appeared/")